In [30]:
#import csv as pd
import pandas as pd

#read csv file
df = pd.read_csv('roman-churches.csv')

#print the first 5 rows
print(df.head().to_markdown(index=False))

|   cid | name                                     | address                          | architect            |   artists | built               |   caption | clergy                   |   consecrated | dedication                         | denomination                           | englishname                        |   fax |   image |   national | phone        |   titular | type       | url                          |
|------:|:-----------------------------------------|:---------------------------------|:---------------------|----------:|:--------------------|----------:|:-------------------------|--------------:|:-----------------------------------|:---------------------------------------|:-----------------------------------|------:|--------:|-----------:|:-------------|----------:|:-----------|:-----------------------------|
|  1500 | All Saints                               | 153/b Via del Babuino 00187 Roma | George Edmund Street |       nan | 1880&ndash;1887     |       nan | Church of

In [31]:
#Drop columns with excessive missing values (e.g., caption, fax, image, url).
df = df.drop(columns=['caption', 'fax', 'image', 'url', 'clergy', 'national', 'phone', 'titular', 'type'])

#print the first 5 rows
print(df.head().to_markdown(index=False))

|   cid | name                                     | address                          | architect            |   artists | built               |   consecrated | dedication                         | denomination                           | englishname                        |
|------:|:-----------------------------------------|:---------------------------------|:---------------------|----------:|:--------------------|--------------:|:-----------------------------------|:---------------------------------------|:-----------------------------------|
|  1500 | All Saints                               | 153/b Via del Babuino 00187 Roma | George Edmund Street |       nan | 1880&ndash;1887     |           nan | nan                                | Anglican Communion / Church of England | All Saints                         |
|  3128 | Angelo Custode                           | Via del Tritone (present site)   | nan                  |       nan | 1624                |           nan | Guardian An

In [32]:
# Convert columns to appropriate data types
df = df.astype({
    'cid': 'int64',                      # Unique identifier as integer
    'name': 'string',                    # Church name as string
    'built': 'string',                   # Original built information as string
    'address': 'string',                 # Address as string
    'architect': 'string',               # Architect's name as string
    'artists': 'string',                 # Artists involved as string
    'consecrated': 'string',             # Consecration year, kept as string
    'dedication': 'string',              # Dedication as string
    'denomination': 'string',            # Denomination as string
    'englishname': 'string',             # English name as string
})

# Verify column data types
print(df.dtypes)

# Print the first 5 rows in a readable format
print(df.head().to_markdown(index=False))

cid                      int64
name            string[python]
address         string[python]
architect       string[python]
artists         string[python]
built           string[python]
consecrated     string[python]
dedication      string[python]
denomination    string[python]
englishname     string[python]
dtype: object
|   cid | name                                     | address                          | architect            | artists   | built               | consecrated   | dedication                         | denomination                           | englishname                        |
|------:|:-----------------------------------------|:---------------------------------|:---------------------|:----------|:--------------------|:--------------|:-----------------------------------|:---------------------------------------|:-----------------------------------|
|  1500 | All Saints                               | 153/b Via del Babuino 00187 Roma | George Edmund Street | <NA>      | 1

In [33]:
import pandas as pd

# Replace '?' with 'Unknown' in the DataFrame
df = df.replace('?', 'Unknown')

# Fill missing values in non-numeric columns with 'Unknown'
for column in df.columns:
    if df[column].dtype == 'object' or df[column].dtype.name == 'string':
        df[column] = df[column].fillna("Unknown")

# Print the first 5 rows
#print(df.head().to_markdown(index=False))

# Check for any remaining missing values in the entire DataFrame
print("Missing values in the table:")
print(df.isna().sum())



Missing values in the table:
cid             0
name            0
address         0
architect       0
artists         0
built           0
consecrated     0
dedication      0
denomination    0
englishname     0
dtype: int64


In [34]:
import re
import pandas as pd
import numpy as np

# Function to clean and convert 'built' into centuries
def convert_to_century(value):
    if pd.isnull(value):  # Handle missing values
        return np.nan
    
    value = str(value).lower().strip()  # Convert to string, lowercase, and strip spaces
    
    # Case 1: Extract digits before "th", "st", "nd", or "rd" (e.g., 20th century -> 20)
    match = re.search(r'(\d+)(?=th|st|nd|rd)', value)
    if match:
        return int(match.group(1))  # Already a century
    
    # Case 2: Extract 4-digit years and convert them to centuries
    match = re.search(r'\b(\d{4})\b', value)
    if match:
        year = int(match.group(1))
        return (year // 100) + 1  # Convert year to century
    
    # Case 3: Extract other standalone digits (e.g., 8th -> 8)
    match = re.search(r'\b(\d{1,2})\b', value)
    if match:
        return int(match.group(1))  # Already a century
    
    # Return NaN if nothing matches
    return np.nan

# Apply the function to the 'built' column
df['built_century'] = df['built'].apply(convert_to_century)

# Ensure built_century is a nullable integer type
df['built_century'] = df['built_century'].astype('Int64')

# Verify results
print("Unique values in 'built_century':", df['built_century'].unique())
# Count rows with NA in 'built_century'
na_count_before = df['built_century'].isna().sum()

print(f"Number of rows with NA in 'built_century' before dropping: {na_count_before}")

# Drop rows with NA in 'built_century'
df = df.dropna(subset=['built_century'])

# Count rows with NA in 'built_century' after dropping
na_count_after = df['built_century'].isna().sum()

print(f"Number of rows with NA in 'built_century' after dropping: {na_count_after}")

total_rows = len(df)
na_percentage = (na_count_before / total_rows) * 100

print(f"Percentage of rows with NA in 'built_century' before dropping: {na_percentage:.2f}%")


Unique values in 'built_century': <IntegerArray>
[  19,   17,   20,   18,    4,    5, <NA>,   21,   15,   16,    3,   13,    8,
    9,   12,   11,   14,    7,    6,    2,   10,    1,   27]
Length: 23, dtype: Int64
Number of rows with NA in 'built_century' before dropping: 105
Number of rows with NA in 'built_century' after dropping: 0
Percentage of rows with NA in 'built_century' before dropping: 11.27%


In [35]:
#Strip extra spaces or special symbols from all columns.
df = df.map(lambda x: str(x).strip() if isinstance(x, str) else x)

#print the first 5 rows
print(df.head().to_markdown(index=False))


|   cid | name                                     | address                          | architect            | artists   | built               | consecrated   | dedication                         | denomination                           | englishname                        |   built_century |
|------:|:-----------------------------------------|:---------------------------------|:---------------------|:----------|:--------------------|:--------------|:-----------------------------------|:---------------------------------------|:-----------------------------------|----------------:|
|  1500 | All Saints                               | 153/b Via del Babuino 00187 Roma | George Edmund Street | Unknown   | 1880&ndash;1887     | Unknown       | Unknown                            | Anglican Communion / Church of England | All Saints                         |              19 |
|  3128 | Angelo Custode                           | Via del Tritone (present site)   | Unknown              | Unknown

In [36]:
#normalization

# Split into two DataFrames
table1 = df[['cid', 'name', 'englishname', 'built', 'built_century', 'address']].copy()
table2 = df[['cid', 'architect', 'artists', 'consecrated', 'dedication', 'denomination']].copy()
# Table 3: RomanPopulation
roman_population = pd.read_csv("romanPopulation.csv")

# Verify the splits
print("Table 1:")
print(table1.head().to_markdown(index=False))

print("\nTable 2:")
print(table2.head().to_markdown(index=False))

print("\nTable 3:")
print(roman_population.head().to_markdown(index=False))



Table 1:
|   cid | name                                     | englishname                        | built               |   built_century | address                          |
|------:|:-----------------------------------------|:-----------------------------------|:--------------------|----------------:|:---------------------------------|
|  1500 | All Saints                               | All Saints                         | 1880&ndash;1887     |              19 | 153/b Via del Babuino 00187 Roma |
|  3128 | Angelo Custode                           | Guardian Angel                     | 1624                |              17 | Via del Tritone (present site)   |
|  4880 | Annunciazione della Villa Flaminia       | Annunciation at the Villa Flaminia | 1960's              |              20 | Viale del Vignola 56             |
|  3144 | Ascensione di Nostro Signore Gesù Cristo | Ascension of Our Lord Jesus Christ | 1954                |              20 | Via Manfredonia 5                |
|

In [27]:
from sqlalchemy import create_engine, text
import pandas as pd

# --- Azure SQL Database Connection ---
password = os.environ["AZURE_SQL_PASSWORD"]
username = os.environ["AZURE_SQL_USERNAME"]
server = "marialagerholm.database.windows.net"
database = "lagerholmDB"

# Connection string to Azure SQL Database
azure_connection = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server"
engine = create_engine(azure_connection)


# --- Drop Existing Tables ---
# try:
#     with engine.begin() as conn:
#         conn.execute(text("DROP TABLE IF EXISTS RomanChurches_Details"))
#         conn.execute(text("DROP TABLE IF EXISTS RomanChurches_Main"))
#         conn.execute(text("DROP TABLE IF EXISTS RomanPopulation"))
#         print("Existing tables dropped successfully!")
# except Exception as e:
#     print(f"Error dropping tables: {e}")

# --- Recreate Tables ---
try:
    # Table 1: RomanChurches_Main
    table1.to_sql('RomanChurches_Main', con=engine, if_exists='replace', index=False)
    with engine.begin() as conn:
        conn.execute(text("ALTER TABLE RomanChurches_Main ALTER COLUMN cid INT NOT NULL"))
        conn.execute(text("ALTER TABLE RomanChurches_Main ADD CONSTRAINT PK_RomanChurches_Main PRIMARY KEY (cid)"))

    # Table 2: RomanChurches_Details
    table2.to_sql('RomanChurches_Details', con=engine, if_exists='replace', index=False)
    with engine.begin() as conn:
        conn.execute(text("ALTER TABLE RomanChurches_Details ALTER COLUMN cid INT NOT NULL"))
        conn.execute(text("""
            ALTER TABLE RomanChurches_Details 
            ADD CONSTRAINT FK_RomanChurches_Details 
            FOREIGN KEY (cid) REFERENCES RomanChurches_Main(cid)
        """))

    # Table 3: RomanPopulation
    roman_population.to_sql('RomanPopulation', con=engine, if_exists='replace', index=False)

    print("Uploaded successfully to Azure!")
except Exception as e:
    print(f"Error uploading tables: {e}")


Existing tables dropped successfully!
Correct tables and data uploaded successfully to Azure!


In [37]:
# Query the database for the count of churches per built_century
query = text("""
    SELECT built_century, COUNT(*) AS count
    FROM RomanChurches_Main
    GROUP BY built_century
""")

df = pd.read_sql_query(query, engine)

import plotly.express as px
import pandas as pd

# Ensure all centuries from 1 to 21 are represented
centuries = list(range(1, 22))
df_full = pd.DataFrame({'built_century': centuries})
df_plot = df_full.merge(df, on='built_century', how='left').fillna(0)

# Create the bar plot
fig = px.bar(
    df_plot,
    x='built_century',
    y='count',
    labels={'built_century': 'Century', 'count': 'Number of Churches'},
    title='Number of churches built in Rome per century',
    text='count'
)

# Update the layout for better visibility and resize the plot
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    xaxis=dict(tickmode='linear', dtick=1),
    yaxis_title='Number of Churches',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=800,
    height=600
)

fig.show()

In [38]:
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine

# Load data from the RomanPopulation table in SQL
query = "SELECT Century, Event, Population FROM RomanPopulation"
df = pd.read_sql(query, con=engine)

# Convert centuries to integers
df['Century'] = df['Century'].astype(int)

# Filter out centuries beyond the 21st century
df = df[df['Century'] <= 21]

# Ensure all centuries from -8 to 21 are represented
centuries = list(range(-8, 22))
df_full = pd.DataFrame({'Century': centuries})

# Merge known population data; many centuries may be missing measurements
df_merged = pd.merge(df_full, df[['Century', 'Population']], on='Century', how='left')
# Interpolate missing population values linearly
df_merged['Population'] = df_merged['Population'].interpolate(method='linear')

# Merge the events back. We'll do this separately to avoid losing event info.
# Since events may only be defined for certain centuries, we just join them back on Century.
df_events = df[['Century', 'Event']].dropna()  # Only rows with events
df_plot = pd.merge(df_merged, df_events, on='Century', how='left')
df_plot['Event'] = df_plot['Event'].fillna('')

# Sort by century
df_plot = df_plot.sort_values('Century')

# Create a subset for events
events_df = df_plot[df_plot['Event'] != ''].copy()
events_df['Event_ID'] = range(1, len(events_df) + 1)

# Merge back into main df to have the Event_ID available
df_plot = df_plot.merge(events_df[['Century', 'Event', 'Event_ID']], on=['Century','Event'], how='left')

# Create the line plot with interpolated values
fig = px.line(
    df_plot,
    x='Century',
    y='Population',
    title='Population of Rome per century (interpolated) https://en.wikipedia.org/wiki/Rome',
    labels={'Population': 'Population', 'Century': 'Century'},
    markers=True,
    line_shape='spline'
)

# Add annotations on the graph with just the ID for events
for i, row in df_plot.iterrows():
    if pd.notnull(row['Event_ID']):
        fig.add_annotation(
            x=row['Century'],
            y=row['Population'],
            text=str(int(row['Event_ID'])),
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-50,
            font=dict(size=14, color='black'),
            arrowcolor='gray',
            xanchor='center',
            yanchor='top'
        )

# Prepare the legend as bullet points
event_list_formatted = "<br>".join([f"• {int(eID)}: {event}" for eID, event in zip(events_df['Event_ID'], events_df['Event'])])

# Add the annotation in the top-left corner with a white background box
fig.add_annotation(
    x=0.01,
    y=0.99,
    xref='paper',
    yref='paper',
    text="<b>Event Legend:</b><br>" + event_list_formatted,
    showarrow=False,
    font=dict(size=10),
    align='left',
    bordercolor='black',
    borderwidth=1,
    bgcolor='white',
    opacity=0.9
)

# Update layout for better visibility
max_pop = df_plot['Population'].max()
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        dtick=1,
        title='Century',
        range=[-8, 22]
    ),
    yaxis=dict(
        title='Population',
        tick0=0,
        dtick=500000,
        range=[0, max_pop * 1.3]
    ),
    width=800,
    height=600,
    margin=dict(l=50, r=50, t=50, b=100),
    title=dict(x=0.5, xanchor='center'),
    font=dict(family="Arial", size=10)
)

fig.show()


In [40]:
from sqlalchemy import text
import pandas as pd
import plotly.express as px

# Query the database for churches dedicated to Mary, excluding those after 21st century
query = text("""
    SELECT m.built_century, COUNT(*) AS count
    FROM RomanChurches_Main m
    JOIN RomanChurches_Details d ON m.cid = d.cid
    WHERE d.dedication LIKE '%Mary%'
      AND m.built_century <= 21
    GROUP BY m.built_century
    ORDER BY m.built_century
""")

df = pd.read_sql_query(query, engine)

# Create a bar plot for churches dedicated to Mary by century (up to 21st century)
fig = px.bar(
    df,
    x='built_century',
    y='count',
    labels={'built_century': 'Century', 'count': 'Number of Churches Dedicated to Virgin Mary'},
    title='Number of churches dedicated to Virgin Mary per century',
    text='count'
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    xaxis=dict(tickmode='linear', dtick=1),
    yaxis_title='Number of Churches',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=800,
    height=600
)

fig.show()
